In [ ]:
from classes.event_nodes import EventNodes

etn = EventNodes()


id: 0, event_type: started_school
id: 1, event_type: finished_school
id: 2, event_type: started_work
id: 3, event_type: new_family_member
id: 4, event_type: bought_a_house
id: 5, event_type: got_married
------------------
id: 4, event_type: bought_a_house
